In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [2]:
# baseline /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/18-14-45
# framework (simple) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/19-47-58
# framework /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/20-17-31
# framework (parallel) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-11/21-18-11
# baseline (sigmoid) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-12/06-58-58
# framework with pre-trianing: /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-12/12-19-17

# /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-15/11-14-43
# /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-15/12-44-33
 
# 0.7846002791466535 framework no slot /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-16/18-00-52

# 0.7260169600798542 framework slot softmax /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-17/07-38-28
# 0.7364939655101216 framework slot entmax /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-17/07-54-40

# 0.7846002791466535 framework no slot reg_dist /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-17/08-34-51

# 0.7568504676981378 framework with pretrained inception /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-18/05-59-37

# 0.799028975925636 baseline w/o pretrained inception /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-18/07-27-42
# 0.7694478071347788 framework w/o pretrained inception /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-18/07-37-22

# -----------------

# (f1 0.45679, purity 0.9428255262449856) Baseline (on hard dataset) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-21/07-02-53
# (f1 0.848765, purity 0.7803162124360771) Framework (on hard dataset) /home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-21/07-12-48




dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-21/07-12-48")







Global seed set to 42


Fetching configuration...
Loading datamodule...


[nltk_data] Downloading package wordnet to /home/danis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 9218.52it/s]


Len of vocab:  53
Max len of caption:  12
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_task' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_task'])`.
  rank_zero_warn(
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_tie' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ig

In [3]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [4]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

In [5]:
scores = list()

for feature_id in range(10):

    attribute_values = [[0, 0] for _ in range(len(attribute_mapping[0]))]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        feature = model.main.inference(image)[1].cpu().detach().numpy()[0][feature_id]
        # feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(scores).mean()

print(metric)

100%|██████████| 1845/1845 [00:39<00:00, 46.18it/s]

0.7803162124360771


In [6]:
scores

[0.8261718458559056,
 0.735116302637449,
 0.7185295575235271,
 0.7377062360480993,
 0.8435091389918494,
 0.6546390088486022,
 0.7309055624544862,
 0.9652576348034337,
 0.7872868203739112,
 0.8040400168235063]

In [ ]:
0.8822225473241826,
 0.9912966747062876,
 0.899363677568215,
 0.9966992345651436,
 0.9723234369973011,
 0.980378357183282,
 0.9678059755469376,
 0.981712635832305,
 0.8758528343560068,
 0.880599888370194]

In [8]:
metric

0.7803162124360771

In [ ]:
model

LitBaseModel(
  (main): BaseModel(
    (feature_extractor): TorchvisionFeatureExtractor(
      (main): Inception3(
        (Conv2d_1a_3x3): BasicConv2d(
          (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2d_2a_3x3): BasicConv2d(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (Conv2d_2b_3x3): BasicConv2d(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        (Conv2d_3b_1x1): BasicConv2d(
          (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1